# Vaults - Setup

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from mezo.currency_utils import get_token_price, format_musd_currency_columns
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 19
python-dotenv could not parse statement starting at line 20
python-dotenv could not parse statement starting at line 21
python-dotenv could not parse statement starting at line 22
python-dotenv could not parse statement starting at line 23
python-dotenv could not parse statement starting at line 24
python-dotenv could not parse statement starting at line 25
python-dotenv could not parse statement starting at line 26
python-dotenv could not parse statement starting at line 27
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 19
python-dotenv could not parse statement starting at line 20
python-dotenv could not parse statement starting at line 21
python-dotenv could not parse statement 

In [2]:
# Subgraph Setup

AUGUST_VAULT_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/mezo-vaults-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_VAULT_TRANSFERS = """
query vaultTransfers($skip: Int!) {
  transfers(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    from
    to
    value
    transactionHash_
    block_number
  }
}
"""

In [3]:
# Instantiate subgraph

vault = SubgraphClient(
    url = AUGUST_VAULT_SUBGRAPH, 
    headers = SUBGRAPH_HEADERS
)

In [4]:
# Fetch subgraph data
vaults =  vault.fetch_subgraph_data(GET_VAULT_TRANSFERS, 'transfers')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
Fetching transactions with skip=5000...
Fetching transactions with skip=6000...
Fetching transactions with skip=7000...
Fetching transactions with skip=8000...
Fetching transactions with skip=9000...
No more records found.


In [5]:
# Save vaults data df

vaults_df = pd.DataFrame(vaults)
display(vaults_df)

,timestamp_,from,to,value,transactionHash_,block_number
0,1759934505,0xc6e8ec9d72ef4f86fac6ac9aa0dcd65b6f5451d8,0x0000000000000000000000000000000000000000,25552341438139239567,0x6c987da1bd47225a7463c64741350f2332f7942bf304...,3746117
1,1759910510,0x0000000000000000000000000000000000000000,0x64125d5e793b05f0290e4d322a5b05fdd3ec2674,20000000000000000000,0x4fd342ea6f509d4af92aef4afbf8d56d52f4b8c078b4...,3739480
2,1759860887,0x0000000000000000000000000000000000000000,0xe6db30da03905b00dcbe207513e1853b8323d5c1,9827745635008444133,0x67ca9335d7f67ab877b5a286e3708440ad5a668c0d81...,3725750
3,1759852801,0x0000000000000000000000000000000000000000,0x71f57b86c2cba46fe9100a981569257c36edf555,21781263628907737122,0x48149d1eb0b80919b46b4b32ccb2908b0236eb35fa00...,3723510
4,1759852596,0x71f57b86c2cba46fe9100a981569257c36edf555,0x0000000000000000000000000000000000000000,153342466159548971047,0x9c8ae937ea9490c4661290d50695ff9fced7baff5cb6...,3723453
...,...,...,...,...,...,...
8389,1750924920,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1000000000000000000,0xf94447d87f136b744db7d595d441d468fca44162c337...,1257108
8390,1750899066,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,5000000000000000000,0x6a633ddd97ece5fd9c72271c72745a20c75635f93932...,1250310
8391,1750894224,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,10000000000000000000,0xb1414979063a26c67326912e08202b38a096e43456aa...,1249036
8392,1750853682,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1000000000000000000,0xeadd15fb5deaf6f11e2bfd1a9421ae40de826f8a5e3e...,1238184


In [6]:
vaults_df.columns

Index(['timestamp_', 'from', 'to', 'value', 'transactionHash_',
       'block_number'],
      dtype='object')

In [7]:
# Exploration 
vaults_df = format_musd_currency_columns(vaults_df, ['value'])
vaults_df = format_datetimes(vaults_df, ['timestamp_'])

vaults_df.head()

,timestamp_,from,to,value,transactionHash_,block_number
0,2025-10-08,0xc6e8ec9d72ef4f86fac6ac9aa0dcd65b6f5451d8,0x0000000000000000000000000000000000000000,25.552341,0x6c987da1bd47225a7463c64741350f2332f7942bf304...,3746117
1,2025-10-08,0x0000000000000000000000000000000000000000,0x64125d5e793b05f0290e4d322a5b05fdd3ec2674,20.000000,0x4fd342ea6f509d4af92aef4afbf8d56d52f4b8c078b4...,3739480
2,2025-10-07,0x0000000000000000000000000000000000000000,0xe6db30da03905b00dcbe207513e1853b8323d5c1,9.827746,0x67ca9335d7f67ab877b5a286e3708440ad5a668c0d81...,3725750
3,2025-10-07,0x0000000000000000000000000000000000000000,0x71f57b86c2cba46fe9100a981569257c36edf555,21.781264,0x48149d1eb0b80919b46b4b32ccb2908b0236eb35fa00...,3723510
4,2025-10-07,0x71f57b86c2cba46fe9100a981569257c36edf555,0x0000000000000000000000000000000000000000,153.342466,0x9c8ae937ea9490c4661290d50695ff9fced7baff5cb6...,3723453


In [8]:
vaults_df['type'] = 'transfer'  # default value
vaults_df.loc[vaults_df['from'] == '0x0000000000000000000000000000000000000000', 'type'] = 'deposit' 
vaults_df.loc[vaults_df['to'] == '0x0000000000000000000000000000000000000000', 'type'] = 'withdrawal'

# Display the result
display(vaults_df[['from', 'to', 'type']].head(10))
print(f"\nType distribution:")
print(vaults_df['type'].value_counts())

,from,to,type
0,0xc6e8ec9d72ef4f86fac6ac9aa0dcd65b6f5451d8,0x0000000000000000000000000000000000000000,withdrawal
1,0x0000000000000000000000000000000000000000,0x64125d5e793b05f0290e4d322a5b05fdd3ec2674,deposit
2,0x0000000000000000000000000000000000000000,0xe6db30da03905b00dcbe207513e1853b8323d5c1,deposit
3,0x0000000000000000000000000000000000000000,0x71f57b86c2cba46fe9100a981569257c36edf555,deposit
4,0x71f57b86c2cba46fe9100a981569257c36edf555,0x0000000000000000000000000000000000000000,withdrawal
5,0xa98be6f81336a23812bc672e6a936b1b4f3fac64,0x0000000000000000000000000000000000000000,withdrawal
6,0x0000000000000000000000000000000000000000,0xc5232a75366a9b40a616aa606636548db716f038,deposit
7,0x0000000000000000000000000000000000000000,0xb0280b58f541131b29d1b33319cd440a99ea0305,deposit
8,0xb56ee863f444b1d58bd7efec48ee87a8aa3610bc,0x0000000000000000000000000000000000000000,withdrawal
9,0x0000000000000000000000000000000000000000,0xb438dbaae78225eefcdbb04e207a8efb06036a2c,deposit



Type distribution:
type
deposit       7767
withdrawal     491
transfer       136
Name: count, dtype: int64


In [9]:
deposits = vaults_df.loc[vaults_df['from'] == '0x0000000000000000000000000000000000000000']
withdrawals = vaults_df.loc[vaults_df['to'] == '0x0000000000000000000000000000000000000000']

In [ ]:
total_deposit_amt = sum(deposits['value'])
total_withdrawal_amt = sum(withdrawals['value'])
tvl = total_deposit_amt - total_withdrawal_amt

In [ ]:
total_depositors = deposits['to'].nunique()
total_deposits = deposits['to'].count()

In [ ]:
total_withdrawers = withdrawals['from'].nunique()
total_withdrawals = withdrawals['from'].count()

In [ ]:
daily_vault_txns = vaults_df.groupby(['timestamp_']).agg(
    value = ('value', 'sum'),
    transactions = ('count', 'sum'),
    users = ('to', lambda x: x.nunique())
).reset_index().sort_values(by='timestamp_', ascending=False)

In [ ]:
daily_vault_txns

In [ ]:
import requests

vault_contract = "0x221B2D9aD7B994861Af3f4c8A80c86C4aa86Bf53"
url = f"https://api.explorer.mezo.org/api/v2/addresses/{vault_contract}/transactions"
transaction_fees = {}


# Create list to store all transaction data
transactions_list = []
next_page_params = None
page_count = 0
method = 'deposit'

while True:
        page_count += 1
        print(f"Fetching page {page_count}...")

        # Build URL with pagination parameters
        if next_page_params:
                response = requests.get(url, params=next_page_params)
        else:
                response = requests.get(url)

        if response.status_code != 200:
                print(f"Error: {response.status_code} - {response.text}")
                break
                
        data = response.json()
        transactions = data.get("items", [])

        if not transactions:
                print("No more transactions found.")
                break

        print(f"Found {len(transactions)} transactions on page {page_count}")

        for tx in data.get("items", []):
                tx_data = {
                        'timestamp_': tx.get('timestamp'),
                        'method': tx.get('method'),
                        'fee_value': int(tx.get('fee', {}).get('value', 0)) if tx.get('fee') else 0,
                        'has_error': tx.get('has_error_in_internal_txs', False),
                        'from_address': tx.get('from', {}).get('hash') if tx.get('from') else None,
                        'to_address': tx.get('to', {}).get('hash') if tx.get('to') else None,
                        'transactionHash_': tx.get('hash'),
                        'block_number': tx.get('block'),
                }

        # Extract decoded input parameters if available
              # Extract only the first decoded parameter (i=0)
                if tx.get('decoded_input') and tx.get('decoded_input').get('parameters'):
                        parameters = tx.get('decoded_input').get('parameters')
                        if len(parameters) > 0:
                                first_param = parameters[0]
                                tx_data['param_0_name'] = first_param.get('name')
                                tx_data['param_0_value'] = str(first_param.get('value'))
                
                transactions_list.append(tx_data)
          
          # Check for next page
        next_page_params = data.get("next_page_params")
        if not next_page_params:
               print("Reached last page.")
               break

# Create DataFrame
transactions_df = pd.DataFrame(transactions_list)

# Display the DataFrame
print(f"Created DataFrame with {len(transactions_df)} transactions")
print(f"Columns: {list(transactions_df.columns)}")
print(f"Methods found: {transactions_df['method'].value_counts()}")

# Display summary
print(f"\n✅ Created DataFrame with {len(transactions_df)} total transactions")
print(f"📊 Columns: {list(transactions_df.columns)}")
print(f"🔧 Methods found: {transactions_df['method'].value_counts()}")
print(f"📅 Date range: {transactions_df['timestamp_'].min()} to {transactions_df['timestamp_'].max()}")

# Show sample data
print(f"\n🔍 First 5 transactions:")
print(transactions_df.head())

# Show transaction fees by method
print(f"\n💰 Average fees by method:")
fee_by_method = transactions_df.groupby('method')['fee_value'].agg(['count', 'mean','sum']).round(0)
print(fee_by_method)

In [17]:
api_url = f'https://api.explorer.mezo.org/api/v2/addresses/{vault_contract}'
response = requests.get(api_url)
data = response.json()

upMUSD_supply = data.get("token").get('total_supply')
upMUSD_supply =  int(upMUSD_supply) / 1e18
upMUSD_supply

1310238.0359813285